# Aries Basic Controller - Basic Message Example

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

## Setup a listener for the basicmessages topic

This is emitted using PyPubSub when the controller receives a basicmessages webhook from the agent. This happens everytime the agent receives a basicmessage.

Note, in this instance the handler responds to any basic message with a message of it's own.

In [3]:
def messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.messaging.send_message(connection_id, "This is a response from Bob"))
    print("Handle message", payload, connection_id)


message_listener = {
    "handler": messages_handler,
    "topic": "basicmessages"
}

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners([message_listener], defaults=True)

## Verify an Active Connection Exists

This should have been established as part of the setup script between the two agents Alice and Bob.

In [4]:
response = await agent_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
else:
    print("You must create a connection")
    

Results :  [{'initiator': 'self', 'created_at': '2020-07-22 12:18:07.316684Z', 'routing_state': 'none', 'state': 'active', 'accept': 'manual', 'invitation_mode': 'once', 'their_label': 'Alice', 'invitation_key': '6JGzxzrp6HEY53dRWaQxB39NhFB6ppXbcadQNydKvhBG', 'their_did': 'RK9pzFAQnNU1hAhruGuxRZ', 'connection_id': '8f46022a-ea65-4819-9846-d278a3905396', 'updated_at': '2020-07-22 12:18:19.467929Z', 'my_did': 'B1yUnjT1eo9ydRhRf8wRUo'}]
Connection : {'initiator': 'self', 'created_at': '2020-07-22 12:18:07.316684Z', 'routing_state': 'none', 'state': 'active', 'accept': 'manual', 'invitation_mode': 'once', 'their_label': 'Alice', 'invitation_key': '6JGzxzrp6HEY53dRWaQxB39NhFB6ppXbcadQNydKvhBG', 'their_did': 'RK9pzFAQnNU1hAhruGuxRZ', 'connection_id': '8f46022a-ea65-4819-9846-d278a3905396', 'updated_at': '2020-07-22 12:18:19.467929Z', 'my_did': 'B1yUnjT1eo9ydRhRf8wRUo'}
Active Connection ID :  8f46022a-ea65-4819-9846-d278a3905396
Handle message {'connection_id': '8f46022a-ea65-4819-9846-d278a

## Send a Basic Message over DIDComm to Bob

See [aries-rfc](https://github.com/hyperledger/aries-rfcs/tree/master/features/0095-basic-message)

You can send as messages as you want, if you are running the [basic-message tutorial on Alice's notebook](http://localhost:8888/notebooks/basic-message.ipynb#Verify-an-Active-Connection-Exists) these will be received and printed by the message handler.

In [ ]:
basic_message = "Hello from Bob"
response = await agent_controller.messaging.send_message(connection_id, basic_message)
print("BASIC MESSAGE - Bob -> Alice")
print(response)

## End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [5]:
response = await agent_controller.terminate()
print(response)

None
